In [1]:
import os
from glob import glob
from pymystem3 import Mystem

In [10]:
PATH_DD = '/home/al/Projects_My/hakaton_ozon_org/'

In [6]:
# Получение списка стоп слов
stop_words = []
with open(PATH_DD + 'dataset/stop_word_base.txt', 'r') as f:
    for i in f:
        stop_words.append(i.strip())

In [7]:
def get_dir(NAME_START: str) -> list:
    '''Получение списка директорий наборов
    '''
    dd = [ f.path for f in os.scandir(NAME_START) if f.is_dir() ]
    return dd

In [8]:
def func(value):
    for i in range(10):
        value = value.replace(str(i),'')
    value = value.replace('(-)', ' ')
    return value

In [9]:
def get_array_text_with_category(nf: str, m):
    """ Получение массива текстов по категориям

    Args:
        nf (str): _description_
        m (_type_): _description_
    """
    
    ddel = ['*', ',', '-', '/', '%', ';', ')', '+', '.', '..', ':']
    with open(nf, 'r') as i_f:
        all_text = i_f.read()
        all_text = all_text.strip().rstrip('\r\n')
        all_text = func(all_text)
        #.replace('/\n', ' ')
        for i in ddel:
            all_text = all_text.replace(i, ' ')
        all_text = all_text.lower().split()
    sum_arr = []
    for word in all_text:
        # print(word)
        lemmas = m.lemmatize(word)
        # doc = Doc(word)
        # print(doc.tokens)
        # print(doc.sents)
        # print(lemmas)
        # print()
        if lemmas[0] not in stop_words:
            sum_arr.append(lemmas[0])
        #break
    return sum_arr

In [14]:
NAME_START_POINT = PATH_DD + 'dataset_loc/'
list_dir = get_dir(NAME_START_POINT)
# print(list_dir)
list_dir = ['/home/al/Projects_My/hakaton_ozon_org/dataset_loc/чехлы',
            '/home/al/Projects_My/hakaton_ozon_org/dataset_loc/Наушники',
            '/home/al/Projects_My/hakaton_ozon_org/dataset_loc/платья'
]
            
categories = list_dir.copy()
array_txt = [] # описание как предложения из лемматизированных слов
m = Mystem()
kk = 0
catY = []
keys_cat = []
all_words_list = []
for sd in list_dir:
    key_one = sd.split('/')[-1].lower()
    print(key_one)
    keys_cat.append(key_one)
    list_dir_two = get_dir(sd)
    for dt in list_dir_two:
        list_text = glob(dt + '/*.txt')
        #print(list_text)
        #print()
        servis_list = []
        # servis_list.append(key_one)
        for nf in list_text:
            dd = get_array_text_with_category(nf, m)
            servis_list += dd
            all_words_list += dd
            #break
        array_txt.append(' '.join(servis_list))
        catY.append(kk)
        # print()
        # print(array_txt[kk])
    kk += 1
    #break

# set_dict = []
# for key in array_txt:
#     print()
#     print(key)
#     print(array_txt[key])
#     dd = dict(array_txt[key])
#     nab = []
#     for kk in dd:
#         if dd[kk] >=2 and kk not in stop_words:
#             nab.append(kk)
#     set_dict.append(nab)
set_all_words = set(all_words_list)
print(keys_cat)
print(len(array_txt))
print(catY)
print(len(set_all_words))

чехлы
наушники
платья
['чехлы', 'наушники', 'платья']
32
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
1109


In [15]:
array_txt[2]

'смартфон накладка камера противоударный силиконовый накладка камера kreiger mix специально спроектировать samsung galaxy a g серьезный периметр обладать усиленный угол смартфон средство воздушный подушка находиться между угол накладка серьезно смягчать удар падение толщина предусматривать камера бортик вокруг камера защищать линза отдельность упругий полиуретан безопасный мягкий ощупь накладка иметься отверстие шнурок помощь смартфон можно носить шея рука силиконовый накладка камера противоударный общий изготовитель мес samsung galaxy a g samsung galaxy a смартфон фиолетовый код продавец'

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [17]:
X_train, X_test, y_train, y_test = train_test_split(array_txt, catY, test_size=0.20, random_state=42)
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)

In [18]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [19]:
# Метод наивный Байес
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [20]:
X_test_new = vectorizer.transform(X_test)
#print(X_test_new.shape)
X_new_tfidf = tfidf_transformer.transform(X_test_new)
predicted = clf.predict(X_new_tfidf)

In [21]:
kk = 0
for doc, category in zip(y_test, predicted):
    print(doc, category)
    if doc == category: kk +=1
        
print('model ', kk/len(y_test))

2 2
1 1
2 2
1 1
0 0
0 0
2 2
model  1.0


In [8]:
vectorizer.get_feature_names_out()

array(['00', '1031436306', '11', ..., 'юбка', 'являться', 'яркость'],
      dtype=object)

In [9]:
print(X.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 5 ... 0 2 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]


In [48]:
pipe = Pipeline([('count', CountVectorizer(vocabulary=set_all_words)),
                 ('tfid', TfidfTransformer())]).fit(array_txt)
# tf_transformer = TfidfTransformer(use_idf=False).fit(array_txt)
pipe['count']

CountVectorizer(vocabulary={'a', 'aac', 'acs', 'air', 'amoled', 'android',
                            'apparel', 'apple', 'aramid', 'avakyan', 'awesome',
                            'bass', 'best', 'black', 'bluetooth', 'borofone',
                            'boutique', 'broscorp', 'by', 'c', 'camera', 'case',
                            'cavolo', 'chrome', 'club', 'concept', 'ew',
                            'femme', 'fiber', 'g', ...})

In [49]:
pipe.transform(array_txt).shape

(32, 1109)

In [50]:
pipe['tfid'].idf_

array([4.49650756, 3.1102132 , 3.80336038, ..., 4.49650756, 4.49650756,
       4.49650756])

In [41]:
import sklearn.datasets as ds
list_tem = dir(ds)
len(list_tem)
print(list_tem)

['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__getattr__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_arff_parser', '_base', '_california_housing', '_covtype', '_kddcup99', '_lfw', '_olivetti_faces', '_openml', '_rcv1', '_samples_generator', '_species_distributions', '_svmlight_format_fast', '_svmlight_format_io', '_twenty_newsgroups', 'clear_data_home', 'descr', 'dump_svmlight_file', 'fetch_20newsgroups', 'fetch_20newsgroups_vectorized', 'fetch_california_housing', 'fetch_covtype', 'fetch_kddcup99', 'fetch_lfw_pairs', 'fetch_lfw_people', 'fetch_olivetti_faces', 'fetch_openml', 'fetch_rcv1', 'fetch_species_distributions', 'get_data_home', 'load_breast_cancer', 'load_diabetes', 'load_digits', 'load_files', 'load_iris', 'load_linnerud', 'load_sample_image', 'load_sample_images', 'load_svmlight_file', 'load_svmlight_files', 'load_wine', 'make_biclusters', 'make_blobs', 'make_checkerboard', 'make_circles', 'make_classification', 'make_friedman1'

In [44]:
from sklearn.datasets import load_iris
from sklearn.datasets import __name__
dd = load_iris()
dd.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [46]:
__name__

'sklearn.datasets'

In [9]:
from sklearn.datasets import fetch_20newsgroups

In [10]:
categories = [
    "alt.atheism",
    "talk.religion.misc",
]

# data_train = fetch_20newsgroups(
#     subset="train",
#     categories=categories,
#     shuffle=True,
#     random_state=42,
#     remove=("headers", "footers", "quotes"),
# )
data_all = fetch_20newsgroups()

In [11]:
data_all.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [13]:
data_all.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [14]:
data_all.target

array([7, 4, 4, ..., 3, 1, 8])

In [19]:
print(data_all.data[8])

From: holmes7000@iscsvax.uni.edu
Subject: WIn 3.0 ICON HELP PLEASE!
Organization: University of Northern Iowa
Lines: 10

I have win 3.0 and downloaded several icons and BMP's but I can't figure out
how to change the "wallpaper" or use the icons.  Any help would be appreciated.


Thanx,

-Brando

PS Please E-mail me




In [18]:
data_train.keys() #['data'][0]

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [19]:
data_train['target']

array([0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [20]:
data_train['target_names']

['alt.atheism', 'talk.religion.misc']

In [ ]:

data_test = fetch_20newsgroups(
    subset="test",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=("headers", "footers", "quotes"),
)

print(f"Loading 20 newsgroups dataset for {len(data_train.target_names)} categories:")
print(data_train.target_names)
print(f"{len(data_train.data)} documents")